In [30]:
#Pickle
import pickle

#Preprocessing
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from keras_preprocessing.image import ImageDataGenerator

#Modeling
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import MobileNetV2

#Data Visualisation
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

#Data Manipulation
import numpy as np
import pandas as pd 

#Python Native
import os
from math import ceil

In [31]:
dataset_path = "/home/sergei/Downloads/.ipynb_checkpoints/archive/fashion-dataset"

In [32]:
df = pd.read_csv(dataset_path+"/styles.csv",nrows=5000, error_bad_lines=False)

In [33]:
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.sample(frac=1).reset_index(drop=True)

In [34]:
image_generator = ImageDataGenerator(
    validation_split=0.2
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path + "/images",
    x_col="image",
    y_col="subCategory",
    target_size=(28,28),
    batch_size=32,
    subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=dataset_path+ "/images",
    x_col="image",
    y_col="subCategory",
    target_size=(28,28),
    batch_size=32,
    subset="validation"
)

classes = len(training_generator.class_indices)

Found 4000 validated image filenames belonging to 36 classes.
Found 1000 validated image filenames belonging to 36 classes.


In [35]:
base_model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(classes, activation='softmax')(x)

In [36]:
model = Model(inputs=base_model.input, outputs=predictions)

In [37]:
for layer in base_model.layers:
    layer.trainable = False

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 48, 48, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [39]:
model.fit_generator(
    generator=training_generator,
    steps_per_epoch=ceil(0.8 * (df.size / 32)),

    validation_data=validation_generator,
    validation_steps=ceil(0.2 * (df.size / 32)),

    epochs=1,
    verbose=1
)

/home/sergei/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1375/1375 [==============================] - 122s 87ms/step - loss: nan - accuracy: 0.0030 - val_loss: nan - val_accuracy: 0.0030


In [52]:
os.mkdir("Model MobileNet")

In [54]:
model.save("Model MobileNet")

/home/sergei/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: Model MobileNet/assets


In [26]:
!git status

On branch Sergei
nothing to commit, working tree clean


In [27]:
!git commit -m "Add MobileNetV2"

On branch Sergei
nothing to commit, working tree clean


In [28]:
!git push origin Sergei

Everything up-to-date
